In [16]:
import pandas as pd
import os
import re

In [3]:
df_sensitive = pd.read_csv("sensitive_topics_vs_source.csv")
df_sensitive_toloka = df_sensitive[df_sensitive['source'] == 'toloka']
toloka_trtes_set = set(df_sensitive[df_sensitive['source']=='trtest']['text'])

In [7]:
topics = list(df_sensitive.columns[1:-1])

In [143]:
topics

['offline_crime',
 'online_crime',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'health_shaming',
 'politics',
 'racism',
 'religion',
 'sexual_minorities',
 'sexism',
 'social_injustice']

In [11]:
'OUTPUT:dsfds'[7:]

'dsfds'

In [153]:
old2new_topic = {'crime_real':'offline_crime','crime_web':'online_crime','halth_shaming':'health_shaming',
                'social':'social_injustice','sex_minorities':'sexual_minorities'}

In [214]:
def update_batch_dict(df_hc, thrsh, collected_texts_dict, file):
    confidnece_columns = []
    output_columns = []
    for c in df_hc.columns:
        if 'CONFIDENCE' in c:
            df_hc[c] = df_hc[c].apply(lambda x: float(x[:-1])/100)
            confidnece_columns.append(c)
        if 'OUTPUT' in c:
            output_columns.append(c)
    
    meet_req_list = []
    for i, el in df_hc.iterrows():
        confidence_above_threshold = all([el[c] > thrsh for c in confidnece_columns if c not in ('CONFIDENCE:none',
                                                                                             'CONFIDENCE:obscene',
                                                                                             'CONFIDENCE:toxic')])
        
#         confidence_above_threshold = [el[c] > thrsh for c in confidnece_columns if c not in ('CONFIDENCE:none',
#                                                                                              'CONFIDENCE:obscene',
#                                                                                              'CONFIDENCE:toxic')]
        
        collected_topics_conf = [0] * 18
        for confidence_colname, output_colname in zip(confidnece_columns, output_columns):
            short_topic = confidence_colname[11:]
            assert short_topic in output_colname
            output_value = el[output_colname]
            confidence_value = el[confidence_colname] if output_value == True else 1 - el[confidence_colname]
            if short_topic in ('none','obscene','toxic'):
                continue
            short_topic = old2new_topic.get(short_topic, short_topic)
            
            topic_index = topics.index(short_topic)
            collected_topics_conf[topic_index] = confidence_value

        if el['INPUT:text'] in collected_texts_dict:
            if collected_texts_dict[el['INPUT:text']]['is_high_conf'] != False:#false нельзя исправлять. если один раз не соотве то все
                collected_texts_dict[el['INPUT:text']]['is_high_conf'] = confidence_above_threshold 
            new_topics_list = []
            for existing_values, values_to_add in zip(collected_texts_dict[el['INPUT:text']]['topics_conf'], collected_topics_conf):
                if existing_values > 0:
                    assert values_to_add == 0, "{} {}".format(i, el['text'])

                new_topics_list.append(existing_values+values_to_add)  
            new_topics_list = [round(v,3) for v in new_topics_list]
            assert len(new_topics_list) == 18
            collected_texts_dict[el['INPUT:text']]['topics_conf'] = new_topics_list
            collected_texts_dict[el['INPUT:text']]['file'].append(file)
            
        else:
            collected_texts_dict[el['INPUT:text']] = {}
            collected_texts_dict[el['INPUT:text']]['topics_conf'] = [round(v,3) for v in collected_topics_conf]
            collected_texts_dict[el['INPUT:text']]['is_high_conf'] = confidence_above_threshold
            collected_texts_dict[el['INPUT:text']]['file'] = [file]
            
    return set([k for k,v in collected_texts_dict.items() if v['is_high_conf'] == True])
    
def turn_batch_into_pd(batch_list):
    pass
    
def get_aggr_batch_with_filtering(batch_name, drop_unconfident_thrsh = None, 
                                  drop_traintest_examples = False, print_output = False, drop_trtest = True):
    if batch_name == 'first': 
        raw_files = ['g1r.tsv','g2r.tsv','g3r.tsv']
    elif batch_name == 'fourth':
        raw_files = ['g1_4r.tsv','g2_4r.tsv','g3_4r.tsv']
    elif batch_name == 'old':
        raw_files = ['maybe_old/g1r.tsv','maybe_old/g2r.tsv']
    
    engaged_texts= set()
    
    collected_texts_dict = {}
   
#     collected_agreements = []

    for i, file in enumerate(raw_files):
        print(file)
        batch_path = "./directly_from_toloka/"+file
        df_curr_common= pd.read_csv(batch_path, sep = '\t')
        
        if print_output == True: print("before dropping trtest ...", len(df_curr_common))
        df_curr_common = df_curr_common[~df_curr_common['INPUT:text'].isin(toloka_trtes_set)]#!!
        if print_output == True: print("after dropping trtest ...", len(df_curr_common))
        
        
        file_agg = re.sub('r','',file)
        
        batch_path_agg = "./directly_from_toloka/"+file_agg
        print(batch_path_agg)
        df_curr_agg = pd.read_csv(batch_path_agg, sep = '\t')

        current_high_conf_texts = update_batch_dict(df_curr_agg, drop_unconfident_thrsh, collected_texts_dict, file_agg)
    return collected_texts_dict

#         if print_output == True: print("before dropping low conf ... ", len(df_curr_common))
#         df_curr_common = df_curr_common[df_curr_common['INPUT:text'].isin(current_high_conf_texts)]
#         if print_output == True: print("after dropping low conf ...", len(df_curr_common))

dct = get_aggr_batch_with_filtering("old", drop_unconfident_thrsh = 0.9)

maybe_old/g1r.tsv
./directly_from_toloka/maybe_old/g1.tsv
maybe_old/g2r.tsv
./directly_from_toloka/maybe_old/g2.tsv


In [211]:
len(dct)

3379

In [212]:
2211

2211

In [213]:
import random
index_r = random.randint(1,len(dct))
index_r = 2211
print(index_r)
index_r_text = list(dct.keys())[index_r]
print(index_r_text)
for c, t in zip(dct[index_r_text]['topics_conf'], topics):
    print(t,c)
print(dct[index_r_text]['is_high_conf'])
print(dct[index_r_text]['file'])

2211
или баба просто сидит и стебется над тупыми мужланами
offline_crime 0.0
online_crime 0.009
drugs 0.0
gambling 0.001
pornography 0.0
prostitution 0.003
slavery 0.0
suicide 0.007
terrorism 0.013
weapons 0.0
body_shaming 0
health_shaming 0
politics 0
racism 0
religion 0
sexual_minorities 0
sexism 0
social_injustice 0
[True, True, True, True, True, True, True, True, True, True]
['maybe_old/g1.tsv']


In [163]:
#first 2394